In [1]:
import pandas as pd

df = pd.read_csv('data/coordenadas/zonificacion_coordenadas_cafe.csv')
df.head()

,id,cod_depart,departamen,cod_dane_m,municipio,lon,lat,area_ha,aptitud
0,1,5,Antioquia,5001,Medellín,-75.611288,6.258531,1133.151245,Aptitud alta
1,2,5,Antioquia,5002,Abejorral,-75.438474,5.804174,13323.429688,No apta
2,3,5,Antioquia,5004,Abriaquí,-76.085735,6.626307,29114.949219,Exclusión legal
3,4,5,Antioquia,5021,Alejandría,-75.090177,6.364923,2455.476562,No apta
4,5,5,Antioquia,5030,Amagá,-75.707902,6.032773,355.028381,Exclusión legal


In [1]:
import requests
import pandas as pd
from pathlib import Path
import time, random
from tqdm import tqdm

# Configuraciones generales
COORDS_CSV = "data/coordenadas/zonificacion_coordenadas_cafe.csv"
OUT_DIR = Path("data/nasa_power")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Departamentos a conservar
KEEP_DEPARTS = {63, 25}  # Quindío y Cundinamarca

# Rango de años a consultar
START_YEAR = 2014
END_YEAR   = 2025  # solo hasta julio 2025

# Configuración API NASA POWER
NASA_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"

PARAMETERS = ",".join([
    "T2M","RH2M","ALLSKY_SFC_SW_DWN","PRECTOTCORR",
    "CLOUD_AMT","WS2M","GWETPROF","V50M","TS_ADJ","TQV"
])

API_CONFIG = {
    "community": "AG",
    "format": "JSON",
    "units": "metric",
    "time-standard": "UTC"
}

# Funciones 
def load_coords():
    df = pd.read_csv(COORDS_CSV)
    df.columns = [c.lower() for c in df.columns]
    return df[df["cod_depart"].isin(KEEP_DEPARTS)]

def call_nasa(lat, lon, start, end):
    params = {
        "start": start,
        "end": end,
        "latitude": lat,
        "longitude": lon,
        "parameters": PARAMETERS,
        **API_CONFIG
    }
    r = requests.get(NASA_URL, params=params, timeout=120)
    r.raise_for_status()
    return r.json()

def parse_json(data):
    times = list(data["properties"]["parameter"]["T2M"].keys())
    rows = []
    for t in times:
        row = {"date": t}
        for var in PARAMETERS.split(","):
            row[var] = data["properties"]["parameter"][var].get(t)
        rows.append(row)
    df = pd.DataFrame(rows)
    df["date"] = pd.to_datetime(df["date"])
    return df


# Proceso principal
df_coords = load_coords()

for _, row in tqdm(df_coords.iterrows(), total=len(df_coords)):
    uid, lat, lon = row["id"], row["lat"], row["lon"]
    out = OUT_DIR / f"{uid}_nasa_power.csv"
    
    if out.exists():
        print("ya existe:", uid)
        continue

    full_df = []
    print(f"\nID {uid} ({lat},{lon})")

    # Consulta por año
    for year in range(START_YEAR, END_YEAR + 1):

        # Validar rangos de fechas que hay datos almacenados
        
        if year == 2025:
            start, end = "20250101", "20250731"  # hasta julio 31 de 2025
        else:
            start, end = f"{year}0101", f"{year}1231"

        try:
            print(f"   → {year} ({start} a {end})")
            j = call_nasa(lat, lon, start, end)
            df_year = parse_json(j)
            full_df.append(df_year)
            time.sleep(random.uniform(2, 3))
        except Exception as e:
            print("Error:", e)

    if full_df:
        final = pd.concat(full_df)
        final.to_csv(out, index=False)
        print("Guardado:", out)

  0%|          | 0/129 [00:00<?, ?it/s]


ID 149 (4.316707855240076,-74.18107344315469)
   → 2014 (20140101 a 20141231)
   → 2015 (20150101 a 20151231)
   → 2016 (20160101 a 20161231)
   → 2017 (20170101 a 20171231)
   → 2018 (20180101 a 20181231)
   → 2019 (20190101 a 20191231)
   → 2020 (20200101 a 20201231)
   → 2021 (20210101 a 20211231)
   → 2022 (20220101 a 20221231)
   → 2023 (20230101 a 20231231)
   → 2024 (20240101 a 20241231)
   → 2025 (20250101 a 20250731)


  1%|          | 1/129 [00:55<1:58:33, 55.58s/it]

Guardado: data/nasa_power/149_nasa_power.csv

ID 459 (4.372601518177892,-74.67078936139487)
   → 2014 (20140101 a 20141231)
   → 2015 (20150101 a 20151231)
   → 2016 (20160101 a 20161231)
   → 2017 (20170101 a 20171231)
   → 2018 (20180101 a 20181231)
   → 2019 (20190101 a 20191231)
   → 2020 (20200101 a 20201231)
   → 2021 (20210101 a 20211231)
   → 2022 (20220101 a 20221231)
   → 2023 (20230101 a 20231231)
   → 2024 (20240101 a 20241231)
   → 2025 (20250101 a 20250731)


  2%|▏         | 2/129 [01:53<2:00:24, 56.88s/it]

...

Guardado: data/nasa_power/829_nasa_power.csv

ID 830 (4.609884089056924,-75.79202492130982)
   → 2014 (20140101 a 20141231)
   → 2015 (20150101 a 20151231)
   → 2016 (20160101 a 20161231)
   → 2017 (20170101 a 20171231)
   → 2018 (20180101 a 20181231)
   → 2019 (20190101 a 20191231)
   → 2020 (20200101 a 20201231)
   → 2021 (20210101 a 20211231)
   → 2022 (20220101 a 20221231)
   → 2023 (20230101 a 20231231)
   → 2024 (20240101 a 20241231)
   → 2025 (20250101 a 20250731)


 99%|█████████▉| 128/129 [2:00:05<00:56, 56.67s/it]

Guardado: data/nasa_power/830_nasa_power.csv

ID 831 (4.611298665048576,-75.53067395463131)
   → 2014 (20140101 a 20141231)
   → 2015 (20150101 a 20151231)
   → 2016 (20160101 a 20161231)
   → 2017 (20170101 a 20171231)
   → 2018 (20180101 a 20181231)
   → 2019 (20190101 a 20191231)
   → 2020 (20200101 a 20201231)
   → 2021 (20210101 a 20211231)
   → 2022 (20220101 a 20221231)
   → 2023 (20230101 a 20231231)
   → 2024 (20240101 a 20241231)
   → 2025 (20250101 a 20250731)


100%|██████████| 129/129 [2:00:59<00:00, 56.28s/it]

Guardado: data/nasa_power/831_nasa_power.csv
